# The Prompt

Let's see how to customize the prompt.

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.insert(0, "../")

In [2]:
import math
import langchain
from kork import (
    CodeChain,
    ast,
    AstPrinter,
    c_,
    r_,
    run_interpreter,
)
from langchain import PromptTemplate
from kork import SimpleContextRetriever

The code below defines a place holder model for a chat model.
Feel free to replace it with a real language model.

In [3]:
from typing import Any, List, Optional

from langchain.chat_models.base import BaseChatModel
from langchain.schema import AIMessage, BaseMessage, ChatGeneration, ChatResult
from pydantic import Extra


class ToyChatModel(BaseChatModel):
    response: str

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    def _generate(
        self, messages: List[BaseMessage], stop: Optional[List[str]] = None
    ) -> ChatResult:
        message = AIMessage(content=self.response)
        generation = ChatGeneration(message=message)
        return ChatResult(generations=[generation])

    async def _agenerate(
        self, messages: List[BaseMessage], stop: Optional[List[str]] = None
    ) -> Any:
        """Async version of _generate."""
        message = AIMessage(content=self.response)
        generation = ChatGeneration(message=message)
        return ChatResult(generations=[generation])

## Custom prompt

Prompts take two optional input variables: the `language_name` and the `external_funcitons_block`. If the prompt uses such a variable, the variable will be auto-populated by the chain.

In [4]:
instruction_template = PromptTemplate(
    template="""
You are coding in a language called the `{language_name}`.



{external_functions_block}

Begin!\n
""",
    input_variables=["language_name", "external_functions_block"],
)

## Declare the chain

In [5]:
chain = CodeChain.from_defaults(
    llm=ToyChatModel(response="MEOW MEOW MEOW MEOW"),  # The LLM to use
    examples=[
        ("2**5", r_(c_(math.pow, 2, 5))),
        ("take the log base 2 of 2", r_(c_(math.log2, 2))),
    ],  # Example programs
    context=[math.pow, math.log2, math.log10],
    instruction_template=instruction_template,
)

In [6]:
environment, few_shot_prompt = chain.prepare_context(query="hello")

In [7]:
environment.list_external_functions()

[ExternFunctionDef(name='pow', params=ParamList(params=[Param(name='x', type_='Any'), Param(name='y', type_='Any')]), return_type='Any', implementation=<built-in function pow>, doc_string='Return x**y (x to the power of y).'),
 ExternFunctionDef(name='log2', params=ParamList(params=[Param(name='x', type_='Any')]), return_type='Any', implementation=<built-in function log2>, doc_string='Return the base 2 logarithm of x.'),
 ExternFunctionDef(name='log10', params=ParamList(params=[Param(name='x', type_='Any')]), return_type='Any', implementation=<built-in function log10>, doc_string='Return the base 10 logarithm of x.')]

In [8]:
print(few_shot_prompt.format_prompt(query="[user input]").to_string())


You are coding in a language called the `😼`.



You have access to the following external functions:

```😼
extern fn pow(x: Any, y: Any) -> Any // Return x**y (x to the power of y).
extern fn log2(x: Any) -> Any // Return the base 2 logarithm of x.
extern fn log10(x: Any) -> Any // Return the base 10 logarithm of x.
```


Begin!

Input: ```text
2**5
```

Output: <code>var result = pow(2, 5)</code>

Input: ```text
take the log base 2 of 2
```

Output: <code>var result = log2(2)</code>

Input: [user input]

Output:
